# MPM Comparison and Migration

This notebook provides tools to:
1. Compare MPM deployments to find common and unique report actions
2. Migrate (copy) report actions between deployments with proper attribute adjustments

## Key Considerations:
- Deployment-specific attributes: `domain_code`, `deployment_version`, `deployment_code` (schema name)
- Communities lists need adjustment for target deployment
- Schedules differ between domains
- Parent dependencies need validation

In [1]:
# Imports
import pandas as pd
import sqlite3
import json
from pathlib import Path
from typing import Dict, List, Set, Tuple
from dataclasses import dataclass

# Setup paths
BASE_DIR = Path.cwd()
DB_PATH = BASE_DIR / "resources" / "meta-db" / "schema-sentinel.db"

print(f"Database: {DB_PATH}")
print(f"Exists: {DB_PATH.exists()}")

Database: /Users/igor.gladyshev/PycharmProjects/schema-sentinel/resources/meta-db/schema-sentinel.db
Exists: True


In [2]:
# Load all data from SQLite
conn = sqlite3.connect(str(DB_PATH))

deployments_df = pd.read_sql_query("SELECT * FROM mpm_deployments", conn)
communities_df = pd.read_sql_query("SELECT * FROM mpm_communities", conn)
sensor_actions_df = pd.read_sql_query("SELECT * FROM mpm_sensor_actions", conn)
report_actions_df = pd.read_sql_query("SELECT * FROM mpm_report_actions", conn)

conn.close()

print("Data loaded:")
print(f"  Deployments: {len(deployments_df)} rows")
print(f"  Communities: {len(communities_df)} rows")
print(f"  Sensor Actions: {len(sensor_actions_df)} rows")
print(f"  Report Actions: {len(report_actions_df)} rows")

print("\nAvailable domains:")
print(deployments_df[['DOMAIN_CODE', 'DEPLOYMENT_VERSION']].to_string(index=False))

Data loaded:
  Deployments: 4 rows
  Communities: 10 rows
  Sensor Actions: 71 rows
  Report Actions: 320 rows

Available domains:
DOMAIN_CODE DEPLOYMENT_VERSION
         AZ              0.0.5
         BS              0.0.5
         CO              0.0.5
         WY              0.0.5


## 1. Compare Report Actions Between Domains

In [3]:
def compare_report_actions(domain1: str, domain2: str, df: pd.DataFrame) -> Dict:
    """
    Compare report actions between two domains.

    Returns:
        Dictionary with:
        - common: action_codes in both domains
        - only_in_domain1: action_codes only in domain1
        - only_in_domain2: action_codes only in domain2
        - common_details: DataFrame with side-by-side comparison
    """
    df1 = df[df['DOMAIN_CODE'] == domain1].copy()
    df2 = df[df['DOMAIN_CODE'] == domain2].copy()

    actions1 = set(df1['ACTION_CODE'].unique())
    actions2 = set(df2['ACTION_CODE'].unique())

    common = actions1 & actions2
    only_in_1 = actions1 - actions2
    only_in_2 = actions2 - actions1

    # Create side-by-side comparison for common actions
    common_details = []
    for action_code in sorted(common):
        row1 = df1[df1['ACTION_CODE'] == action_code].iloc[0]
        row2 = df2[df2['ACTION_CODE'] == action_code].iloc[0]

        common_details.append({
            'ACTION_CODE': action_code,
            f'{domain1}_REPORT_NAME': row1['REPORT_NAME'],
            f'{domain2}_REPORT_NAME': row2['REPORT_NAME'],
            f'{domain1}_COMMUNITIES': row1['COMMUNITIES'],
            f'{domain2}_COMMUNITIES': row2['COMMUNITIES'],
            f'{domain1}_SCHEDULE': row1['SCHEDULE'],
            f'{domain2}_SCHEDULE': row2['SCHEDULE'],
            'NAMES_MATCH': row1['REPORT_NAME'] == row2['REPORT_NAME'],
            'SCHEDULES_MATCH': row1['SCHEDULE'] == row2['SCHEDULE'],
        })

    return {
        'common': sorted(common),
        'only_in_domain1': sorted(only_in_1),
        'only_in_domain2': sorted(only_in_2),
        'common_details': pd.DataFrame(common_details),
        'count_common': len(common),
        'count_only_domain1': len(only_in_1),
        'count_only_domain2': len(only_in_2),
    }

# Example: Compare AZ and BS
comparison = compare_report_actions('AZ', 'BS', report_actions_df)

print(f"\n{'='*80}")
print(f"Comparison: AZ vs BS")
print(f"{'='*80}")
print(f"Common report actions: {comparison['count_common']}")
print(f"Only in AZ: {comparison['count_only_domain1']}")
print(f"Only in BS: {comparison['count_only_domain2']}")

print(f"\nCommon actions:")
for action in comparison['common'][:10]:  # Show first 10
    print(f"  - {action}")
if len(comparison['common']) > 10:
    print(f"  ... and {len(comparison['common']) - 10} more")

print(f"\nOnly in AZ (sample):")
for action in comparison['only_in_domain1'][:10]:
    print(f"  - {action}")
if len(comparison['only_in_domain1']) > 10:
    print(f"  ... and {len(comparison['only_in_domain1']) - 10} more")

print(f"\nOnly in BS (sample):")
for action in comparison['only_in_domain2'][:10]:
    print(f"  - {action}")
if len(comparison['only_in_domain2']) > 10:
    print(f"  ... and {len(comparison['only_in_domain2']) - 10} more")


Comparison: AZ vs BS
Common report actions: 58
Only in AZ: 52
Only in BS: 3

Common actions:
  - retail_liability_ticket_report
  - retail_liability_ticket_report_per_community
  - retail_recap_all_report
  - retail_recap_all_report_month_to_date
  - retail_recap_report_per_community
  - retail_sports_pool_cancelled_ticket_report
  - retail_sports_pool_cancelled_ticket_report_month_to_date
  - retail_sports_pool_cancelled_ticket_report_month_to_date_per_community
  - retail_sports_pool_cancelled_ticket_report_per_community
  - retail_sports_pool_expired_ticket_report
  ... and 48 more

Only in AZ (sample):
  - 3k_report_bop
  - bop_cash_deposit_withdrawal_transaction_report
  - daily_transaction_deposit_withdrawal_report_bop
  - daily_transaction_report_bop
  - dormant_acct_report_bop
  - dtr_deposit_failed_backin_report
  - forfeited_bonus_winnings_report_bop
  - liability_ticket_report_bop
  - month_to_date_transaction_deposit_withdrawal_report_bop
  - noncashable_bonus_balance_repo

In [4]:
# View detailed comparison for common actions
print("Common actions - detailed comparison:")
print("\nActions with different names:")
diff_names = comparison['common_details'][~comparison['common_details']['NAMES_MATCH']]
print(diff_names[['ACTION_CODE', 'AZ_REPORT_NAME', 'BS_REPORT_NAME']].to_string(index=False))

print("\nActions with different schedules:")
diff_schedules = comparison['common_details'][~comparison['common_details']['SCHEDULES_MATCH']]
print(f"Count: {len(diff_schedules)}")
if len(diff_schedules) > 0:
    print(diff_schedules[['ACTION_CODE', 'AZ_SCHEDULE', 'BS_SCHEDULE']].head(5).to_string(index=False))

Common actions - detailed comparison:

Actions with different names:
                                             ACTION_CODE                                AZ_REPORT_NAME                                           BS_REPORT_NAME
            retail_liability_ticket_report_per_community    Retail Sports Pool Liability Ticket Report Retail Sports Pool Liability Ticket Report Per Community
                   retail_recap_all_report_month_to_date Retail Sports Pool Recap Report Month To Date            Retail Sports Pool Recap Month To Date Report
                       retail_recap_report_per_community                           Retail Recap Report            Retail Sports Pool Recap Report Per Community
retail_sports_pool_cancelled_ticket_report_per_community           Retail Sports Pool Cancelled Report        Retail Sports Pool Cancelled Report Per Community
    retail_sports_pool_resettlement_report_per_community        Retail Sports Pool Resettlement Report     Retail Sports Pool Reset

## 2. Comparison Matrix - All Domains

In [5]:
# Create comparison matrix for all domain pairs
domains = deployments_df['DOMAIN_CODE'].unique()

print("Report Actions Comparison Matrix:")
print("="*80)

matrix = []
for d1 in domains:
    row = {'Domain': d1}
    for d2 in domains:
        if d1 == d2:
            total = len(report_actions_df[report_actions_df['DOMAIN_CODE'] == d1])
            row[d2] = f"{total} (total)"
        else:
            comp = compare_report_actions(d1, d2, report_actions_df)
            row[d2] = f"{comp['count_common']} common"
    matrix.append(row)

matrix_df = pd.DataFrame(matrix)
print(matrix_df.to_string(index=False))

Report Actions Comparison Matrix:
Domain          AZ         BS          CO         WY
    AZ 110 (total)  58 common  101 common  40 common
    BS   58 common 61 (total)   58 common   0 common
    CO  101 common  58 common 107 (total)  40 common
    WY   40 common   0 common   40 common 42 (total)


## 3. Migration Helper - Copy Report Actions

In [6]:
@dataclass
class MigrationConfig:
    """Configuration for migrating a report action between domains."""
    source_domain: str
    target_domain: str
    action_code: str
    # Optional overrides
    new_schedule: str = None
    new_communities: List[str] = None
    adjust_parents: bool = True

def prepare_action_migration(
    config: MigrationConfig,
    report_actions_df: pd.DataFrame,
    deployments_df: pd.DataFrame,
    communities_df: pd.DataFrame
) -> Dict:
    """
    Prepare a report action for migration from source to target domain.

    Returns:
        Dictionary with:
        - original: Original action data
        - migrated: Adjusted action data for target domain
        - changes: List of changes made
        - warnings: List of warnings/issues to address
    """
    # Get source action
    source_action = report_actions_df[
        (report_actions_df['DOMAIN_CODE'] == config.source_domain) &
        (report_actions_df['ACTION_CODE'] == config.action_code)
    ]

    if len(source_action) == 0:
        raise ValueError(f"Action {config.action_code} not found in {config.source_domain}")

    source_action = source_action.iloc[0].to_dict()

    # Get target deployment info
    target_deployment = deployments_df[
        deployments_df['DOMAIN_CODE'] == config.target_domain
    ].iloc[0]

    # Get target communities
    target_communities = communities_df[
        communities_df['DOMAIN_CODE'] == config.target_domain
    ]['COMMUNITY_ID'].tolist()

    # Create migrated action
    migrated = source_action.copy()
    changes = []
    warnings = []

    # 1. Update deployment-specific attributes
    migrated['DOMAIN_CODE'] = config.target_domain
    changes.append(f"DOMAIN_CODE: {source_action['DOMAIN_CODE']} → {config.target_domain}")

    migrated['DEPLOYMENT_VERSION'] = target_deployment['DEPLOYMENT_VERSION']
    changes.append(f"DEPLOYMENT_VERSION: {source_action['DEPLOYMENT_VERSION']} → {target_deployment['DEPLOYMENT_VERSION']}")

    # 2. Adjust communities
    try:
        communities = json.loads(source_action['COMMUNITIES']) if source_action['COMMUNITIES'] else []

        if config.new_communities is not None:
            # Use specified communities
            new_communities = config.new_communities
            changes.append(f"COMMUNITIES: {len(communities)} → {len(new_communities)} (manually specified)")
        elif communities:
            # Check if source communities exist in target
            valid_communities = [c for c in communities if c in target_communities]
            invalid_communities = [c for c in communities if c not in target_communities]

            if invalid_communities:
                warnings.append(f"Communities not in target: {invalid_communities}")
                warnings.append(f"Valid target communities: {target_communities}")

            new_communities = valid_communities
            changes.append(f"COMMUNITIES: {len(communities)} → {len(valid_communities)} (filtered for target)")
        else:
            new_communities = []

        migrated['COMMUNITIES'] = json.dumps(new_communities)
    except (json.JSONDecodeError, TypeError) as e:
        warnings.append(f"Failed to parse COMMUNITIES: {e}")

    # 3. Adjust schedule
    if config.new_schedule:
        migrated['SCHEDULE'] = config.new_schedule
        changes.append(f"SCHEDULE: Updated with provided schedule")
    else:
        warnings.append("SCHEDULE: Review required - may need timezone/timing adjustments")

    # 4. Check parents
    try:
        parents = json.loads(source_action['PARENTS']) if source_action['PARENTS'] else []
        if parents:
            # Check if parents exist in target domain
            target_actions = set(report_actions_df[
                report_actions_df['DOMAIN_CODE'] == config.target_domain
            ]['ACTION_CODE'].tolist())

            missing_parents = [p for p in parents if p not in target_actions]
            if missing_parents:
                warnings.append(f"Parent actions not in target: {missing_parents}")
                if config.adjust_parents:
                    valid_parents = [p for p in parents if p in target_actions]
                    migrated['PARENTS'] = json.dumps(valid_parents)
                    changes.append(f"PARENTS: Removed missing parents - {len(parents)} → {len(valid_parents)}")
    except (json.JSONDecodeError, TypeError) as e:
        warnings.append(f"Failed to parse PARENTS: {e}")

    # 5. Update query reference schema names
    try:
        query_ref = json.loads(source_action['QUERY_REFERENCE']) if source_action['QUERY_REFERENCE'] else {}
        if query_ref and 'database_name' in query_ref:
            # This might need schema name adjustment
            warnings.append(f"QUERY_REFERENCE: Review database_name '{query_ref['database_name']}' - may need schema adjustment")
    except (json.JSONDecodeError, TypeError) as e:
        warnings.append(f"Failed to parse QUERY_REFERENCE: {e}")

    return {
        'original': source_action,
        'migrated': migrated,
        'changes': changes,
        'warnings': warnings,
    }

print("Migration helper functions loaded.")

Migration helper functions loaded.


## 4. Example: Prepare Migration

In [7]:
# Example: Migrate a report action from AZ to WY
# First, find a common action to use as example
az_actions = set(report_actions_df[report_actions_df['DOMAIN_CODE'] == 'AZ']['ACTION_CODE'])
example_action = list(az_actions)[0] if az_actions else None

if example_action:
    print(f"Example: Migrating action '{example_action}' from AZ to WY\n")

    config = MigrationConfig(
        source_domain='AZ',
        target_domain='WY',
        action_code=example_action,
        adjust_parents=True
    )

    result = prepare_action_migration(
        config,
        report_actions_df,
        deployments_df,
        communities_df
    )

    print("Changes to be made:")
    for change in result['changes']:
        print(f"  ✓ {change}")

    if result['warnings']:
        print("\nWarnings - Manual Review Required:")
        for warning in result['warnings']:
            print(f"  ⚠️  {warning}")

    print("\nOriginal action details:")
    print(f"  ACTION_CODE: {result['original']['ACTION_CODE']}")
    print(f"  REPORT_NAME: {result['original']['REPORT_NAME']}")
    print(f"  DOMAIN_CODE: {result['original']['DOMAIN_CODE']}")
    print(f"  COMMUNITIES: {result['original']['COMMUNITIES']}")

    print("\nMigrated action details:")
    print(f"  ACTION_CODE: {result['migrated']['ACTION_CODE']}")
    print(f"  REPORT_NAME: {result['migrated']['REPORT_NAME']}")
    print(f"  DOMAIN_CODE: {result['migrated']['DOMAIN_CODE']}")
    print(f"  COMMUNITIES: {result['migrated']['COMMUNITIES']}")
else:
    print("No actions found to use as example")

Example: Migrating action 'rosi_bill_denominations_per_community' from AZ to WY

Changes to be made:
  ✓ DOMAIN_CODE: AZ → WY
  ✓ DEPLOYMENT_VERSION: 0.0.5 → 0.0.5
  ✓ COMMUNITIES: 3 → 0 (filtered for target)
  ✓ PARENTS: Removed missing parents - 1 → 0

Warnings - Manual Review Required:
  ⚠️  Communities not in target: ['Chase_Field', 'Harrahs_Ak_Chin', 'Arizona_Downs']
  ⚠️  Valid target communities: []
  ⚠️  SCHEDULE: Review required - may need timezone/timing adjustments
  ⚠️  Parent actions not in target: ['rosi_bill_denominations_all']
  ⚠️  QUERY_REFERENCE: Review database_name 'NJ_Prod' - may need schema adjustment

Original action details:
  ACTION_CODE: rosi_bill_denominations_per_community
  REPORT_NAME: Rosi Bill Denominations Report
  DOMAIN_CODE: AZ
  COMMUNITIES: [
  "Chase_Field",
  "Harrahs_Ak_Chin",
  "Arizona_Downs"
]

Migrated action details:
  ACTION_CODE: rosi_bill_denominations_per_community
  REPORT_NAME: Rosi Bill Denominations Report
  DOMAIN_CODE: WY
  COMMU

## 5. Batch Analysis - Actions to Consider Migrating

In [8]:
def analyze_migration_candidates(
    source_domain: str,
    target_domain: str,
    report_actions_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Analyze which actions from source could be migrated to target.
    """
    comparison = compare_report_actions(source_domain, target_domain, report_actions_df)

    # Actions only in source are candidates
    candidates = report_actions_df[
        (report_actions_df['DOMAIN_CODE'] == source_domain) &
        (report_actions_df['ACTION_CODE'].isin(comparison['only_in_domain1']))
    ][['ACTION_CODE', 'REPORT_NAME', 'ABBREVIATION', 'ACTION_TYPE']].copy()

    candidates['SOURCE'] = source_domain
    candidates['TARGET'] = target_domain

    return candidates

# Example: What could we migrate from AZ to WY?
candidates = analyze_migration_candidates('AZ', 'WY', report_actions_df)

print(f"\nActions in AZ that could be migrated to WY: {len(candidates)}")
print("\nSample candidates:")
print(candidates.head(20).to_string(index=False))


Actions in AZ that could be migrated to WY: 70

Sample candidates:
                                                           ACTION_CODE                                                                    REPORT_NAME ABBREVIATION ACTION_TYPE SOURCE TARGET
                                                         3k_report_bop                                    Deposit Withdrawal {retail_property} Report          KRB      REPORT     AZ     WY
                        bop_cash_deposit_withdrawal_transaction_report                                     Cash Deposit Withdrawal Transaction Report       BCDWTR      REPORT     AZ     WY
                    noncashable_bonus_balance_report_bop_month_to_date                                Non-cashable Bonus Balance Month To Date Report     NBBRBMTD      REPORT     AZ     WY
                                            parlay_card_trading_report                                                     Parlay Card Trading Report         PCTR      REPORT  

## 6. Export Migration Plan

In [9]:
# Create a migration plan for review
def create_migration_plan(
    source_domain: str,
    target_domain: str,
    action_codes: List[str],
    report_actions_df: pd.DataFrame,
    deployments_df: pd.DataFrame,
    communities_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Create a migration plan for multiple actions.
    """
    plan = []

    for action_code in action_codes:
        try:
            config = MigrationConfig(
                source_domain=source_domain,
                target_domain=target_domain,
                action_code=action_code,
                adjust_parents=True
            )

            result = prepare_action_migration(
                config,
                report_actions_df,
                deployments_df,
                communities_df
            )

            plan.append({
                'ACTION_CODE': action_code,
                'REPORT_NAME': result['original']['REPORT_NAME'],
                'SOURCE': source_domain,
                'TARGET': target_domain,
                'CHANGES_COUNT': len(result['changes']),
                'WARNINGS_COUNT': len(result['warnings']),
                'STATUS': 'READY' if len(result['warnings']) == 0 else 'REVIEW_REQUIRED',
                'WARNINGS': ' | '.join(result['warnings']) if result['warnings'] else '',
            })
        except Exception as e:
            plan.append({
                'ACTION_CODE': action_code,
                'REPORT_NAME': '',
                'SOURCE': source_domain,
                'TARGET': target_domain,
                'CHANGES_COUNT': 0,
                'WARNINGS_COUNT': 1,
                'STATUS': 'ERROR',
                'WARNINGS': str(e),
            })

    return pd.DataFrame(plan)

# Example: Create migration plan for first 5 candidates
sample_actions = candidates['ACTION_CODE'].head(5).tolist()

if sample_actions:
    migration_plan = create_migration_plan(
        'AZ', 'WY',
        sample_actions,
        report_actions_df,
        deployments_df,
        communities_df
    )

    print("\nMigration Plan Summary:")
    print(migration_plan[['ACTION_CODE', 'REPORT_NAME', 'STATUS', 'WARNINGS_COUNT']].to_string(index=False))

    # Save to CSV for review
    output_path = BASE_DIR / "migration_plan_AZ_to_WY.csv"
    migration_plan.to_csv(output_path, index=False)
    print(f"\nFull migration plan saved to: {output_path}")


Migration Plan Summary:
                                       ACTION_CODE                                     REPORT_NAME          STATUS  WARNINGS_COUNT
                                     3k_report_bop     Deposit Withdrawal {retail_property} Report REVIEW_REQUIRED               5
    bop_cash_deposit_withdrawal_transaction_report      Cash Deposit Withdrawal Transaction Report REVIEW_REQUIRED               3
noncashable_bonus_balance_report_bop_month_to_date Non-cashable Bonus Balance Month To Date Report REVIEW_REQUIRED               2
                        parlay_card_trading_report                      Parlay Card Trading Report REVIEW_REQUIRED               3
                    retail_liability_ticket_report      Retail Sports Pool Liability Ticket Report REVIEW_REQUIRED               3

Full migration plan saved to: /Users/igor.gladyshev/PycharmProjects/schema-sentinel/migration_plan_AZ_to_WY.csv


## 7. Generic Table Comparison Framework

Using YAML Shredder output or any table structure

In [ ]:
from typing import Optional

@dataclass
class TableConfig:
    """Configuration for a table to compare."""
    name: str
    primary_key: str
    grouping_column: str  # e.g., DOMAIN_CODE
    display_columns: List[str]  # Columns to show in comparison

class GenericTableComparer:
    """Generic tool to compare data across groups in any table."""

    def __init__(self, db_path: Path):
        """Initialize with database connection."""
        self.db_path = db_path
        self.conn = None

    def connect(self):
        """Connect to database."""
        self.conn = sqlite3.connect(str(self.db_path))

    def disconnect(self):
        """Close database connection."""
        if self.conn:
            self.conn.close()
            self.conn = None

    def discover_tables(self) -> List[str]:
        """Discover all tables in database."""
        if not self.conn:
            self.connect()

        query = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
        tables = pd.read_sql_query(query, self.conn)
        return tables['name'].tolist()

    def analyze_table_structure(self, table_name: str) -> Dict:
        """Analyze table structure to suggest comparison configuration."""
        if not self.conn:
            self.connect()

        # Get column info
        query = f'PRAGMA table_info("{table_name}")'
        columns_df = pd.read_sql_query(query, self.conn)

        # Get sample data
        sample_query = f'SELECT * FROM "{table_name}" LIMIT 5'
        sample_df = pd.read_sql_query(sample_query, self.conn)

        # Identify potential key columns
        id_columns = [col for col in columns_df['name'] if 'id' in col.lower()]
        pk_column = columns_df[columns_df['pk'] == 1]['name'].tolist()

        # Identify potential grouping columns
        grouping_candidates = []
        for col in sample_df.columns:
            if sample_df[col].dtype == 'object':
                unique_count = sample_df[col].nunique()
                if 2 <= unique_count <= 10:  # Reasonable grouping column
                    grouping_candidates.append(col)

        return {
            'table_name': table_name,
            'row_count': len(sample_df),
            'columns': columns_df['name'].tolist(),
            'primary_keys': pk_column if pk_column else id_columns,
            'grouping_candidates': grouping_candidates,
            'all_columns_info': columns_df.to_dict('records')
        }

    def compare_groups(
        self,
        table_name: str,
        grouping_column: str,
        comparison_columns: Optional[List[str]] = None,
        primary_key: Optional[str] = None
    ) -> Dict:
        """
        Compare data between groups in a table.

        Args:
            table_name: Name of table to compare
            grouping_column: Column to group by (e.g., DOMAIN_CODE)
            comparison_columns: Columns to compare (None = all columns)
            primary_key: Column to use as identifier

        Returns:
            Dictionary with comparison results
        """
        if not self.conn:
            self.connect()

        # Load data
        df = pd.read_sql_query(f'SELECT * FROM "{table_name}"', self.conn)

        if grouping_column not in df.columns:
            raise ValueError(f"Grouping column '{grouping_column}' not found in table")

        # Identify primary key if not specified
        if not primary_key:
            id_cols = [col for col in df.columns if 'id' in col.lower() or col == 'ACTION_CODE']
            primary_key = id_cols[0] if id_cols else df.columns[0]

        # Get unique groups
        groups = df[grouping_column].unique()

        # Compare each pair of groups
        comparisons = {}
        for i, g1 in enumerate(groups):
            for g2 in groups[i+1:]:
                comp_result = self._compare_two_groups(
                    df, g1, g2, grouping_column, primary_key, comparison_columns
                )
                comparisons[f"{g1}_vs_{g2}"] = comp_result

        return {
            'table': table_name,
            'grouping_column': grouping_column,
            'groups': list(groups),
            'group_counts': df.groupby(grouping_column).size().to_dict(),
            'comparisons': comparisons
        }

    def _compare_two_groups(
        self,
        df: pd.DataFrame,
        group1: str,
        group2: str,
        grouping_column: str,
        primary_key: str,
        comparison_columns: Optional[List[str]]
    ) -> Dict:
        """Compare two specific groups."""
        df1 = df[df[grouping_column] == group1]
        df2 = df[df[grouping_column] == group2]

        ids1 = set(df1[primary_key].unique())
        ids2 = set(df2[primary_key].unique())

        common = ids1 & ids2
        only_in_1 = ids1 - ids2
        only_in_2 = ids2 - ids1

        # Compare values for common records
        differences = []
        if comparison_columns is None:
            comparison_columns = [col for col in df.columns
                                 if col not in [grouping_column, primary_key]]

        for pk_value in common:
            row1 = df1[df1[primary_key] == pk_value].iloc[0]
            row2 = df2[df2[primary_key] == pk_value].iloc[0]

            diffs = {}
            for col in comparison_columns:
                if col in df.columns:
                    val1 = row1[col]
                    val2 = row2[col]
                    if pd.notna(val1) and pd.notna(val2) and val1 != val2:
                        diffs[col] = {'group1': val1, 'group2': val2}

            if diffs:
                differences.append({
                    primary_key: pk_value,
                    'differences': diffs
                })

        return {
            'group1': group1,
            'group2': group2,
            'common_count': len(common),
            'only_in_group1_count': len(only_in_1),
            'only_in_group2_count': len(only_in_2),
            'common_ids': sorted(common),
            'only_in_group1': sorted(only_in_1),
            'only_in_group2': sorted(only_in_2),
            'differences': differences,
            'differences_count': len(differences)
        }

print("Generic table comparer loaded.")

### Example 1: Discover and Analyze Tables

In [ ]:
# Initialize comparer
comparer = GenericTableComparer(DB_PATH)
comparer.connect()

# Discover all tables
tables = comparer.discover_tables()
print("Available tables:")
for table in tables:
    print(f"  - {table}")

# Analyze structure of key tables
print("\n" + "="*80)
print("TABLE STRUCTURE ANALYSIS")
print("="*80)

for table in ['MPM_CONFIG', 'COMMUNITIES', 'ACTIONS']:
    if table in tables:
        print(f"\n{table}:")
        analysis = comparer.analyze_table_structure(table)
        print(f"  Rows: {analysis['row_count']}")
        print(f"  Columns: {len(analysis['columns'])}")
        print(f"  Primary Keys: {analysis['primary_keys']}")
        print(f"  Grouping Candidates: {analysis['grouping_candidates']}")
        print(f"  All Columns: {', '.join(analysis['columns'][:10])}")
        if len(analysis['columns']) > 10:
            print(f"              ... and {len(analysis['columns']) - 10} more")

### Example 2: Compare Old MPM Tables (mpm_report_actions)

In [ ]:
# Compare the old MPM tables if they exist
if 'mpm_report_actions' in tables:
    print("Comparing mpm_report_actions by DOMAIN_CODE:\n")

    comparison = comparer.compare_groups(
        table_name='mpm_report_actions',
        grouping_column='DOMAIN_CODE',
        comparison_columns=['REPORT_NAME', 'SCHEDULE', 'ACTION_TYPE'],
        primary_key='ACTION_CODE'
    )

    print(f"Groups found: {comparison['groups']}")
    print(f"\nGroup sizes:")
    for group, count in comparison['group_counts'].items():
        print(f"  {group}: {count} records")

    # Show comparison for one pair
    if comparison['comparisons']:
        first_comp_key = list(comparison['comparisons'].keys())[0]
        comp_result = comparison['comparisons'][first_comp_key]

        print(f"\n{'-'*80}")
        print(f"Comparison: {comp_result['group1']} vs {comp_result['group2']}")
        print(f"{'-'*80}")
        print(f"Common records: {comp_result['common_count']}")
        print(f"Only in {comp_result['group1']}: {comp_result['only_in_group1_count']}")
        print(f"Only in {comp_result['group2']}: {comp_result['only_in_group2_count']}")
        print(f"Records with differences: {comp_result['differences_count']}")

        if comp_result['differences']:
            print(f"\nSample differences (first 3):")
            for diff in comp_result['differences'][:3]:
                print(f"\n  {diff['ACTION_CODE']}:")
                for col, vals in diff['differences'].items():
                    print(f"    {col}:")
                    print(f"      {comp_result['group1']}: {vals['group1']}")
                    print(f"      {comp_result['group2']}: {vals['group2']}")
else:
    print("mpm_report_actions table not found")

### Example 3: Compare New YAML Shredder Tables (ACTIONS)

In [ ]:
# Compare YAML Shredder output tables
# Note: These tables don't have a grouping column by design - they're from a single file
# But we can still analyze their structure

if 'ACTIONS' in tables:
    print("Analyzing ACTIONS table (YAML Shredder output):\n")

    analysis = comparer.analyze_table_structure('ACTIONS')

    print(f"Table: ACTIONS")
    print(f"  Columns: {len(analysis['columns'])}")
    print(f"  Sample columns: {', '.join(analysis['columns'][:15])}")
    if len(analysis['columns']) > 15:
        print(f"  ... and {len(analysis['columns']) - 15} more")

    # Load data to show summary
    actions_df = pd.read_sql_query('SELECT * FROM ACTIONS', comparer.conn)

    print(f"\nData summary:")
    print(f"  Total actions: {len(actions_df)}")

    # Show action types distribution
    if 'action_type' in actions_df.columns:
        print(f"\nAction types:")
        print(actions_df['action_type'].value_counts().to_string())

    # Show sample records
    print(f"\nSample actions (first 3):")
    display_cols = ['action_code', 'action_type', 'abbreviation']
    available_cols = [col for col in display_cols if col in actions_df.columns]
    if available_cols:
        print(actions_df[available_cols].head(3).to_string(index=False))

    print("\n💡 Note: ACTIONS table is from a single YAML file.")
    print("   To compare across domains, load multiple YAML files with:")
    print("   uv run python yaml_shredder_cli.py all <file> -db <db> -r <domain>_CONFIG")
else:
    print("ACTIONS table not found. Run YAML shredder first:")

### Example 4: Batch Comparison Report

In [ ]:
# Generate comparison report for all relevant tables
def generate_comparison_report(
    comparer: GenericTableComparer,
    tables_to_compare: List[Dict[str, str]]
) -> pd.DataFrame:
    """
    Generate comparison report for multiple tables.

    Args:
        comparer: GenericTableComparer instance
        tables_to_compare: List of dicts with 'table', 'grouping_column', 'primary_key'

    Returns:
        Summary DataFrame
    """
    report = []

    for config in tables_to_compare:
        table = config['table']

        # Check if table exists
        all_tables = comparer.discover_tables()
        if table not in all_tables:
            continue

        try:
            comparison = comparer.compare_groups(
                table_name=table,
                grouping_column=config['grouping_column'],
                primary_key=config.get('primary_key')
            )

            for comp_key, comp_result in comparison['comparisons'].items():
                report.append({
                    'TABLE': table,
                    'COMPARISON': comp_key,
                    'GROUP_1': comp_result['group1'],
                    'GROUP_2': comp_result['group2'],
                    'COMMON': comp_result['common_count'],
                    'ONLY_IN_1': comp_result['only_in_group1_count'],
                    'ONLY_IN_2': comp_result['only_in_group2_count'],
                    'DIFFERENCES': comp_result['differences_count'],
                })
        except Exception as e:
            print(f"Error comparing {table}: {e}")

    return pd.DataFrame(report)

# Configure tables to compare
tables_config = [
    {'table': 'mpm_deployments', 'grouping_column': 'DOMAIN_CODE', 'primary_key': 'DEPLOYMENT_CODE'},
    {'table': 'mpm_communities', 'grouping_column': 'DOMAIN_CODE', 'primary_key': 'COMMUNITY_ID'},
    {'table': 'mpm_sensor_actions', 'grouping_column': 'DOMAIN_CODE', 'primary_key': 'ACTION_CODE'},
    {'table': 'mpm_report_actions', 'grouping_column': 'DOMAIN_CODE', 'primary_key': 'ACTION_CODE'},
]

# Generate report
print("Generating comparison report...\n")
report_df = generate_comparison_report(comparer, tables_config)

if len(report_df) > 0:
    print("COMPARISON REPORT:")
    print("="*100)
    print(report_df.to_string(index=False))

    # Save to CSV
    report_path = BASE_DIR / "table_comparison_report.csv"
    report_df.to_csv(report_path, index=False)
    print(f"\n✓ Report saved to: {report_path}")
else:
    print("No comparisons generated. Check if tables exist in database.")

### Example 5: Export Detailed Comparison to Files

In [ ]:
# Export detailed comparison with differences
def export_detailed_comparison(
    comparer: GenericTableComparer,
    table_name: str,
    grouping_column: str,
    output_dir: Path,
    primary_key: Optional[str] = None
):
    """Export detailed comparison results to CSV files."""
    output_dir.mkdir(exist_ok=True)

    comparison = comparer.compare_groups(
        table_name=table_name,
        grouping_column=grouping_column,
        primary_key=primary_key
    )

    # Export summary
    summary_data = []
    for comp_key, comp_result in comparison['comparisons'].items():
        summary_data.append({
            'Comparison': comp_key,
            'Group 1': comp_result['group1'],
            'Group 2': comp_result['group2'],
            'Common Records': comp_result['common_count'],
            f"Only in {comp_result['group1']}": comp_result['only_in_group1_count'],
            f"Only in {comp_result['group2']}": comp_result['only_in_group2_count'],
            'Records with Differences': comp_result['differences_count'],
        })

    summary_df = pd.DataFrame(summary_data)
    summary_path = output_dir / f"{table_name}_comparison_summary.csv"
    summary_df.to_csv(summary_path, index=False)
    print(f"✓ Summary saved: {summary_path}")

    # Export detailed differences for each comparison
    for comp_key, comp_result in comparison['comparisons'].items():
        if comp_result['differences']:
            diff_data = []
            for diff in comp_result['differences']:
                row = {primary_key or 'ID': diff[primary_key or list(diff.keys())[0]]}
                for col, vals in diff['differences'].items():
                    row[f"{col}_{comp_result['group1']}"] = vals['group1']
                    row[f"{col}_{comp_result['group2']}"] = vals['group2']
                diff_data.append(row)

            diff_df = pd.DataFrame(diff_data)
            diff_path = output_dir / f"{table_name}_{comp_key}_differences.csv"
            diff_df.to_csv(diff_path, index=False)
            print(f"✓ Differences saved: {diff_path}")

# Export comparisons
if 'mpm_report_actions' in comparer.discover_tables():
    output_dir = BASE_DIR / "comparison_exports"

    print("Exporting detailed comparisons...\n")
    export_detailed_comparison(
        comparer,
        table_name='mpm_report_actions',
        grouping_column='DOMAIN_CODE',
        output_dir=output_dir,
        primary_key='ACTION_CODE'
    )

    print(f"\n✓ All exports saved to: {output_dir}")

# Cleanup
comparer.disconnect()